In [ ]:
from pathlib import Path

import pandas as pd

cefr_words = pd.concat(
    [
        (
            pd.read_csv(i)
            .rename(columns={0: "word"})
            .assign(level=i.stem)
            .pipe(lambda x: x[x["word"].str.isalpha()])
        )
        for i in Path("cefr_words").glob("*.csv")
    ]
)
print(cefr_words)

for i in Path("book_vocab").glob("*.csv"):
    df = pd.read_csv(i)
    df = pd.merge(df, cefr_words, on="word", how="left").loc[:, ["word", "level"]]
    df.to_csv(i.parent.parent / "books_vocab_cefr" / i.name, index=False)

In [ ]:
for i in Path("books_vocab_cefr").glob("*.csv"):
    df = pd.read_csv(i)
    df = df.fillna(value="BEYOND")
    df.to_csv(i, index=False)


In [1]:
from pathlib import Path
import pandas as pd
from reading_path_builder import LayeredCEFRBookSelector

books_vocab = {
    book_name.stem: set(pd.read_csv(book_name)["word"])
    for book_name in Path("book_vocab").glob("*.csv")
    # if not any(
    #     x in book_name.stem for x in ["书虫", "译林", "大英百科", "双语", "精讲版"]
    # )
}
vocab_levels = pd.concat(
    [
        (
            pd.read_csv(i, header=None)
            .rename(columns={0: "word"})
            .assign(level=i.stem)
            .pipe(lambda x: x[x["word"].str.isalpha()])
        )
        for i in Path("cefr_words").glob("*.csv")
    ]
)
vocab_levels = {i.word: i.level for i in vocab_levels.itertuples()}

book_selector = LayeredCEFRBookSelector(
    books_vocab=books_vocab, vocab_levels=vocab_levels
)

📚 学习词表分层统计（来自 vocab_levels）:
  A1: 801词
  A2: 733词
  B1: 800词
  B2: 1358词
  C1: 1300词


In [5]:
# 获取多种路径方案

print(
    book_selector.create_progressive_reading_path(
        max_books_per_level={"A1": 7, "A2": 6, "B1": 5, "B2": 3, "C1": 2},
        target_coverage_per_level={
            "A1": 0.95,
            "A2": 0.95,
            "B1": 0.95,
            "B2": 0.95,
            "C1": 0.95,
        },
        max_unknown_ratio=0.4,
        min_relevant_ratio=0.6,
        min_target_level_words=100,
    )
)



=== 选择 A1 等级书籍 ===
  A1等级候选书籍: 6本
目标词汇: 801, 已覆盖: 0, 待覆盖: 801
  选择: lorkq-小公主 | 有声书虫（1级）
  新增A1词汇: 198
  当前覆盖率: 24.7%
  选择: btsbyz-爱情与金钱 | 有声书虫（1级）
  新增A1词汇: 53
  当前覆盖率: 31.3%
  选择: jvvhu-风中奇缘 | 有声书虫（2级）
  新增A1词汇: 22
  当前覆盖率: 34.1%
  选择: bbagn-潘德尔的巫师 | 有声书虫（1级）
  新增A1词汇: 10
  当前覆盖率: 35.3%
  选择: bntzc-阿拉丁和神灯 | 有声书虫（1级）
  新增A1词汇: 4
  当前覆盖率: 35.8%
  选择: gjxdl-猴爪 | 有声书虫（1级）
  新增A1词汇: 2
  当前覆盖率: 36.1%
完成 A1 后累积覆盖率:
  A1: 36.1%
  A2: 14.5%
  B1: 6.8%
  B2: 2.4%
  C1: 0.8%

=== 选择 A2 等级书籍 ===
  A2等级候选书籍: 17本
目标词汇: 733, 已覆盖: 106, 待覆盖: 627
  选择: bsvzaa-三十九级台阶 | 有声书虫（4级）
  新增A2词汇: 164
  当前覆盖率: 22.4%
  选择: efwol-巴斯克维尔猎犬 | 有声书虫（4级）
  新增A2词汇: 52
  当前覆盖率: 29.5%
  选择: xodqi-三怪客泛舟记 | 有声书虫（4级）
  新增A2词汇: 18
  当前覆盖率: 31.9%
  选择: ivqus-小妇人 | 有声书虫（4级）
  新增A2词汇: 13
  当前覆盖率: 33.7%
  选择: cktrg-勃朗特一家的故事 | 有声书虫（3级）
  新增A2词汇: 8
  当前覆盖率: 34.8%
  选择: rxrhp-一个国王的爱情故事 | 有声书虫（2级）
  新增A2词汇: 6
  当前覆盖率: 35.6%
完成 A2 后累积覆盖率:
  A1: 57.7%
  A2: 50.1%
  B1: 31.5%
  B2: 11.8%
  C1: 3.1%

=== 选择 B1 等级书籍 ===
  B1等级候选书籍: 18本
目标

In [ ]:
path = book_selector.create_progressive_reading_path(
    max_books_per_level={"A1": 3, "A2": 3, "B1": 4, "B2": 3, "C1": 3},
    target_coverage_per_level={"A1": 0.85, "A2": 0.9, "B1": 0.9, "B2": 0.9, "C1": 0.9},
    max_unknown_ratio=0.5,
    min_relevant_ratio=0.5,
    min_target_level_words=50,
)

# 输出路径
book_selector.print_reading_path(path, "推荐路径")

In [2]:
# 获取多种路径方案
paths = book_selector.get_alternative_paths()

# 展示所有路径
for path_name, path_result in paths:
    book_selector.print_reading_path(path_result, path_name)


=== 选择 A1 等级书籍 ===
  A1等级候选书籍: 7本
目标词汇: 801, 已覆盖: 0, 待覆盖: 801
  选择: lorkq-小公主 | 有声书虫（1级）
  新增A1词汇: 198
  当前覆盖率: 24.7%
  选择: btsbyz-爱情与金钱 | 有声书虫（1级）
  新增A1词汇: 53
  当前覆盖率: 31.3%
  选择: jvvhu-风中奇缘 | 有声书虫（2级）
  新增A1词汇: 22
  当前覆盖率: 34.1%
  选择: bbagn-潘德尔的巫师 | 有声书虫（1级）
  新增A1词汇: 10
  当前覆盖率: 35.3%
完成 A1 后累积覆盖率:
  A1: 35.3%
  A2: 12.7%
  B1: 5.5%
  B2: 1.8%
  C1: 0.5%

=== 选择 A2 等级书籍 ===
  A2等级候选书籍: 16本
目标词汇: 733, 已覆盖: 93, 待覆盖: 640
  选择: bbvlrq-弗兰肯斯坦 | 有声书虫（3级）
  新增A2词汇: 88
  当前覆盖率: 12.0%
  选择: dlmqa-华特·迪士尼传｜大英百科
  新增A2词汇: 51
  当前覆盖率: 19.0%
  选择: bvdvdp-乔布斯传｜大英百科
  新增A2词汇: 35
  当前覆盖率: 23.7%
  选择: cktrg-勃朗特一家的故事 | 有声书虫（3级）
  新增A2词汇: 27
  当前覆盖率: 27.4%
完成 A2 后累积覆盖率:
  A1: 55.3%
  A2: 40.1%
  B1: 20.8%
  B2: 8.5%
  C1: 2.3%

=== 选择 B1 等级书籍 ===
  B1等级候选书籍: 2本
警告: B1 没有找到合适的候选书籍
完成 B1 后累积覆盖率:
  A1: 55.3%
  A2: 40.1%
  B1: 20.8%
  B2: 8.5%
  C1: 2.3%

=== 选择 B2 等级书籍 ===
  B2等级候选书籍: 0本
警告: B2 没有找到合适的候选书籍
完成 B2 后累积覆盖率:
  A1: 55.3%
  A2: 40.1%
  B1: 20.8%
  B2: 8.5%
  C1: 2.3%

=== 选择 C1 等级书籍 ===
  C1等级候